In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('stopwords')
import os
import fnmatch
import io
from pprint import pprint
from time import time  # to time our operations

#LDA
import warnings
warnings.filterwarnings('ignore')
import os 
%matplotlib inline
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, tokenize
from gensim.models import CoherenceModel

import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/clementnicolas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/anaconda3/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [23]:
City = "Taipei"
NumTopics=11

lda = gensim.models.LdaModel.load('lda_final.{}'.format(City))

In [24]:
np.random.seed(1)


topic1, topic2, topic3, topic4, topic5, topic6, topic7, topic8, topic9,topic10, topic11 = lda.show_topics(11,100)
#, topic10, topic11, topic12, topic13
Topics = [topic1, topic2, topic3, topic4, topic5, topic6, topic7, topic8, topic9,topic10, topic11]

In [25]:
topic6

(5,
 '0.026*"road" + 0.021*"station" + 0.013*"area" + 0.012*"park" + 0.012*"parking" + 0.010*"water" + 0.009*"traffic" + 0.009*"route" + 0.009*"take" + 0.009*"line" + 0.008*"transportation" + 0.008*"public" + 0.007*"operation" + 0.007*"stop" + 0.007*"fire" + 0.007*"period" + 0.007*"bridge" + 0.006*"light" + 0.006*"riverside" + 0.006*"hour" + 0.006*"train" + 0.006*"bicycle" + 0.006*"driver" + 0.006*"street" + 0.006*"measure" + 0.006*"time" + 0.005*"section" + 0.005*"office" + 0.005*"also" + 0.005*"agency" + 0.005*"lane" + 0.005*"vehicle" + 0.005*"cycling" + 0.005*"designated" + 0.005*"department" + 0.005*"exercise" + 0.004*"holiday" + 0.004*"police" + 0.004*"emergency" + 0.004*"year" + 0.004*"located" + 0.004*"minute" + 0.004*"intersection" + 0.004*"reminds" + 0.004*"minzu" + 0.004*"safety" + 0.004*"drill" + 0.004*"avoid" + 0.004*"alley_lane" + 0.004*"bus" + 0.004*"main" + 0.004*"wheel" + 0.003*"closed" + 0.003*"district" + 0.003*"staircase" + 0.003*"starting" + 0.003*"exit" + 0.003*"ce

In [26]:
np.random.seed(2)

from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full

def make_topics_bow(topic):
    # takes the string returned by model.show_topics()
    # split on strings to get topics and the probabilities
    topic = topic.split('+')
    # list to store topic bows
    topic_bow = []
    for word in topic:
        # split probability and word
        prob, word = word.split('*')
        # get rid of spaces
        word = word.split('"')[1]
        word = word.replace(" ",'')
        #print(word)
        # convert to word_type
        word = lda.id2word.doc2bow([word])[0][0]
        topic_bow.append((word, float(prob)))
    return topic_bow


topic_1 = make_topics_bow(topic1[1])
topic_2 = make_topics_bow(topic2[1])
topic_3 = make_topics_bow(topic3[1])
topic_4 = make_topics_bow(topic4[1])

print(hellinger(topic_1, topic_2), hellinger(topic_1, topic_3), hellinger(topic_1, topic_4))

0.6111524905337291 0.7080060856446175 0.7157839585923855


In [27]:
for topicA in Topics:
    for topicB in Topics:
        topic_A = make_topics_bow(topicA[1])
        topic_B = make_topics_bow(topicB[1])
        print(round(hellinger(topic_A,topic_B),2), end = ' ')
    print('')

0.0 0.61 0.71 0.72 0.74 0.64 0.72 0.68 0.68 0.59 0.66 
0.61 0.0 0.67 0.69 0.74 0.62 0.7 0.68 0.64 0.55 0.63 
0.71 0.67 0.0 0.75 0.79 0.65 0.74 0.73 0.68 0.69 0.71 
0.72 0.69 0.75 0.0 0.76 0.72 0.75 0.73 0.7 0.68 0.74 
0.74 0.74 0.79 0.76 0.0 0.71 0.78 0.76 0.73 0.75 0.79 
0.64 0.62 0.65 0.72 0.71 0.0 0.71 0.69 0.66 0.65 0.67 
0.72 0.7 0.74 0.75 0.78 0.71 0.0 0.73 0.72 0.7 0.74 
0.68 0.68 0.73 0.73 0.76 0.69 0.73 0.0 0.71 0.67 0.72 
0.68 0.64 0.68 0.7 0.73 0.66 0.72 0.71 0.0 0.67 0.71 
0.59 0.55 0.69 0.68 0.75 0.65 0.7 0.67 0.67 0.0 0.62 
0.66 0.63 0.71 0.74 0.79 0.67 0.74 0.72 0.71 0.62 0.0 


In [48]:
hellinger(make_topics_bow(topic4[1]), make_topics_bow(topic11[1]))

0.5038763526486121

In [51]:
print(topic1)

(0, '0.025*"city" + 0.015*"mayor" + 0.011*"community" + 0.008*"program" + 0.008*"said" + 0.007*"year" + 0.007*"public" + 0.007*"housing" + 0.006*"work" + 0.005*"resident" + 0.005*"service" + 0.005*"neighborhood" + 0.005*"plan" + 0.005*"opportunity" + 0.005*"development" + 0.004*"project" + 0.004*"support" + 0.004*"people" + 0.004*"youth" + 0.004*"also" + 0.004*"office" + 0.004*"help" + 0.004*"today" + 0.004*"building" + 0.003*"department" + 0.003*"center" + 0.003*"need" + 0.003*"school" + 0.003*"family" + 0.003*"access" + 0.003*"initiative" + 0.003*"space" + 0.002*"make" + 0.002*"goal" + 0.002*"health" + 0.002*"including" + 0.002*"effort" + 0.002*"business" + 0.002*"local" + 0.002*"state" + 0.002*"information" + 0.002*"unit" + 0.002*"partner" + 0.002*"working" + 0.002*"process" + 0.002*"home" + 0.002*"street" + 0.002*"create" + 0.002*"affordable" + 0.002*"part" + 0.002*"partnership" + 0.002*"cultural" + 0.002*"provide" + 0.002*"resource" + 0.002*"art" + 0.002*"member" + 0.002*"organiza

In [42]:
print(topic11)

(10, '0.043*"street" + 0.008*"event" + 0.007*"right_onto" + 0.006*"left_onto" + 0.006*"parking_restrictions" + 0.005*"zone_stopping" + 0.005*"avenue" + 0.004*"traffic" + 0.004*"police_special" + 0.004*"information" + 0.004*"place" + 0.004*"closed_traffic" + 0.003*"effect_follows" + 0.003*"parking" + 0.003*"route" + 0.003*"boylston" + 0.003*"city" + 0.002*"side" + 0.002*"charles" + 0.002*"around"')


In [50]:
for topicA in Topics:
    for topicB in Topics:
        topic_A = make_topics_bow(topicA[1])
        topic_B = make_topics_bow(topicB[1])
        print(round(kullback_leibler(topic_A,topic_B),2), end = ' ')
    print('')

0.0 inf inf inf inf inf inf inf inf inf inf inf inf 
inf 0.0 inf inf inf inf inf inf inf inf inf inf inf 
inf inf 0.0 inf inf inf inf inf inf inf inf inf inf 
inf inf inf 0.0 inf inf inf inf inf inf inf inf inf 
inf inf inf inf 0.0 inf inf inf inf inf inf inf inf 
inf inf inf inf inf 0.0 inf inf inf inf inf inf inf 
inf inf inf inf inf inf 0.0 inf inf inf inf inf inf 
inf inf inf inf inf inf inf 0.0 inf inf inf inf inf 
inf inf inf inf inf inf inf inf 0.0 inf inf inf inf 
inf inf inf inf inf inf inf inf inf 0.0 inf inf inf 
inf inf inf inf inf inf inf inf inf inf 0.0 inf inf 
inf inf inf inf inf inf inf inf inf inf inf 0.0 inf 
inf inf inf inf inf inf inf inf inf inf inf inf 0.0 
